In [ ]:
# Change directory to the root so that relative path loads work correctly
import os

try:
    os.chdir(os.path.join(os.getcwd(), ".."))
    print(os.getcwd())
except:
    pass

In [ ]:
from datetime import datetime

import numpy as np
import torch
from torch import nn

from experiments.C_nonlinear_projection.constraints import (
    helmholtz_equation,
    pythagorean_equation,
    truth_residual,
)
from experiments.C_nonlinear_projection.error_functions import Huber_Error, Lp_Error
from experiments.C_nonlinear_projection.experiment_definition import dictionary_product
from experiments.C_nonlinear_projection.main import run_experiment
from experiments.C_nonlinear_projection.model import Dense, ParameterizedDense, Swish

In [ ]:
# Saving utilities
def get_savefile():
    base_name = "nonlinear-projection"
    time_string = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    savefile = f"{base_name}_{time_string}.pth"
    return savefile


save_directory = os.path.expandvars(
    "$SCRATCH/results/checkpoints/C_nonlinear_projection"
)


def save_out(summary, savefile, directory=save_directory):
    full_file = f"{directory}/{savefile}"
    print(f"Saving to file {full_file}")
    torch.save(summary, full_file)

In [ ]:
base_configuration = {
    "training_parameterizations": {
        "amplitudes": np.linspace(0.2, 5.0, num=10),
        "frequencies": np.linspace(0.2, 5.0, num=10),
        "phases": np.linspace(-0.5, 0.5, num=10),
        "num_points": 500,
        "sampling": "random",
    },
    "testing_parameterizations": {
        "amplitudes": [1.0],
        "frequencies": [1.0],
        "phases": [0.0],
        "num_points": 100,
        "sampling": "uniform",
    },
    "batch_size": 1000,
    "projection_batch_size": 50,
    "architecture": ParameterizedDense,
    "model_size": [50, 50, 50, 50, 50],
    "learning_rate": 1e-3,
    "model_act": Swish(),
    "device": "cpu",
    "constraint": helmholtz_equation,
    "error_fn": None,
    "regularization_weight": 0,
    "tolerance": 0,  # Force that we always use max iterations projection steps
    "max_iterations": 1e4,
}

configuration_revisions = list(
    dictionary_product(
        **{
            "max_iterations": [1e5],
            "tolerance": [1e-3],
            "regularization_weight": [0.0],
            "learning_rate": [1e-3],
            "projection_learning_rate": [1e-4],
        }
    )
)
num_epochs = 100
save_interval = 10

In [ ]:
# Run experiment
all_savefiles = list()
final_checkpoints = list()
for revision in configuration_revisions:
    configuration = base_configuration.copy()
    configuration.update(revision)

    savefile = get_savefile()
    all_savefiles.append(savefile)
    print(f"Running proof of constraint with savefile {savefile}")
    checkpoint_save_file_base = os.path.splitext(savefile)[0]
    final_checkpoints.append(f"{checkpoint_save_file_base}_{num_epochs:05d}.pth")
    final_result = run_experiment(
        num_epochs,
        log=print,
        save_directory=save_directory,
        save_file=checkpoint_save_file_base,
        save_interval=save_interval,
        evaluate=True,
        inference=True,
        **configuration,
    )
    print(f"Completed run with savefile {savefile}")
print("")
print(f"Files were saved to {all_savefiles}")
print(f"Checkpoints were saved to {final_checkpoints}")
print(f"Corresponding revisions {configuration_revisions}")

In [ ]:
print("done!")